In [1]:
import pandas as pd
import datetime
import os
import timeit
from ortools.linear_solver import pywraplp
from pandas import ExcelWriter

load d:\Python3.11\Lib\site-packages\ortools\.libs\zlib1.dll...
load d:\Python3.11\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load d:\Python3.11\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load d:\Python3.11\Lib\site-packages\ortools\.libs\re2.dll...
load d:\Python3.11\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load d:\Python3.11\Lib\site-packages\ortools\.libs\highs.dll...
load d:\Python3.11\Lib\site-packages\ortools\.libs\ortools.dll...


In [2]:
# Load the data
# df2019 = pd.read_csv('/home/kevin/Downloads/BESS/data/raw/2019/merged_df_2019_cleaned.csv')
df2020 = pd.read_csv('D:/Python Projects/git_projects/BESS/data/raw/2020/merged_df_2020_cleaned.csv')
df2021 = pd.read_csv('D:/Python Projects/git_projects/BESS/data/raw/2021/merged_df_2021_cleaned.csv')
df2022 = pd.read_csv('D:/Python Projects/git_projects/BESS/data/raw/2022/merged_df_2022_cleaned.csv')
df2023 = pd.read_csv('D:/Python Projects/git_projects/BESS/data/raw/2023/merged_df_2023_cleaned.csv')
df2024 = pd.read_csv('D:/Python Projects/git_projects/BESS/data/raw/2024/merged_df_2024_cleaned.csv')

In [3]:
df2024.tail()

,datetime_,alberta_internal_load,forecast_alberta_internal_load,pool_price,forecast_pool_price,rolling_30day_avg_price,solar_generation,temp_calgary,temp_edmonton,temp_fortmc,...,alberta_internal_load_lag_15,alberta_internal_load_lag_16,alberta_internal_load_lag_17,alberta_internal_load_lag_18,alberta_internal_load_lag_19,alberta_internal_load_lag_20,alberta_internal_load_lag_21,alberta_internal_load_lag_22,alberta_internal_load_lag_23,alberta_internal_load_lag_24
8782,2024-12-31 20:00:00,11228.0,11227.0,35.340,37.070,26.02,0.000000,-10.5,-13.7,-18.3,...,10397.0,10277.0,10239.0,10280.0,10358.0,10496.0,10732.0,10932.0,11191.0,11326.0
8783,2024-12-31 21:00:00,11107.0,11071.0,33.830,35.300,26.02,0.000000,-10.5,-13.6,-18.0,...,10636.0,10397.0,10277.0,10239.0,10280.0,10358.0,10496.0,10732.0,10932.0,11191.0
8784,2024-12-31 22:00:00,10918.0,10885.0,31.640,31.930,26.02,0.000000,-10.8,-13.7,-18.9,...,10979.0,10636.0,10397.0,10277.0,10239.0,10280.0,10358.0,10496.0,10732.0,10932.0
8785,2024-12-31 23:00:00,10745.0,10684.0,32.370,32.310,26.05,0.004500,-11.5,-13.5,-19.2,...,11252.0,10979.0,10636.0,10397.0,10277.0,10239.0,10280.0,10358.0,10496.0,10732.0
8786,2024-03-10 02:00:00,10078.5,10079.0,30.555,30.305,57.39,11.617639,-0.8,2.5,0.1,...,11345.0,11252.0,10979.0,10636.0,10397.0,10277.0,10239.0,10280.0,10358.0,10496.0


In [4]:
# Concatenate the data
# df2019
df = pd.concat([df2020, df2021, df2022, df2023, df2024], axis=0, ignore_index=True)

In [6]:
df['datetime_'] = pd.to_datetime(df['datetime_'])
df.sort_values(by=["datetime_"], inplace=True)

In [7]:
df=df.iloc[-48:]
df=df[['datetime_', 'pool_price', 'alberta_internal_load', 'wind_generation', 'solar_generation']]
df['datetime_'] = pd.to_datetime(df['datetime_'])

In [8]:
df.head()

,datetime_,pool_price,alberta_internal_load,wind_generation,solar_generation
43814,2024-12-30 00:00:00,36.46,10200.0,627.903359,0.100
43815,2024-12-30 01:00:00,27.10,10118.0,689.852140,0.100
43816,2024-12-30 02:00:00,22.24,10089.0,796.449486,0.100
43817,2024-12-30 03:00:00,20.86,10062.0,830.823290,0.104
43818,2024-12-30 04:00:00,22.13,10078.0,833.325204,0.100


In [91]:
batt_df = {
    "max_charge_rate": [9.0],  # Example value in MW
    "max_discharge_rate": [9.0],  # Example value in MW
    "capacity": [20.0],  # Example value in MWh
    "charge_eff": [0.05],  # Example efficiency
    "discharge_eff": [0.95],  # Example efficiency
    "min_soc": [0.1],  # Minimum state of charge
    "max_soc": [0.95],  # Maximum state of charge
    "initial_soc": [0.5]  # Initial state of charge
}

grid_df = {
    "max_buy_power" : [13000], 
    "max_sell_power" : [13000], 
    "max_import_power" : [13000], 
    "max_export_power" : [13000]
}

battery_df = pd.DataFrame(batt_df)
grid_df = pd.DataFrame(grid_df)


In [92]:
market1DF = df.copy()
market1DF.sort_values(by=["datetime_"], inplace=True)
market1DF["time_string"] = market1DF.apply(
    lambda x: (x["datetime_"] + datetime.timedelta(seconds=0.002)).strftime("%Y-%m-%d %H:%M:%S"), axis=1
)
market1DF.set_index("time_string", inplace=True)
marketDF = market1DF

In [ ]:
#marketDF.drop(columns=["datetime_"], inplace=True)

In [93]:
marketDF.isnull().sum()

datetime_                0
pool_price               0
alberta_internal_load    0
wind_generation          0
solar_generation         0
dtype: int64

In [94]:
marketDict = marketDF.to_dict()
gridDict = grid_df.to_dict()
battDict = battery_df.to_dict()

In [95]:
# Calculate time interval
timeInterval = df.iloc[1]['datetime_'] - df.iloc[0]['datetime_']

In [85]:
timeInterval

Timedelta('0 days 01:00:00')

In [96]:
# Assign the data to the input structure
input_data = type("input", (dict,), {})()
input_data.update({
    "simData": {
        "startTime": datetime.datetime.strptime(marketDF.index[0], "%Y-%m-%d %H:%M:%S"),
        "dt": int(round(timeInterval.total_seconds())) / (60 * 60),  # in hours
        "tIndex": marketDF.shape[0]
    },
    "market": {
        key: {sub_key: sub_item for sub_key, sub_item in marketDict[key].items()}
        for key in marketDict.keys() if key != "datetime_"
    },
    "grid": {key: item[0] for key, item in gridDict.items()},
    "batt": {key: item[0] for key, item in battDict.items()}
})

In [97]:
# Create the mip solver with the CBC backend.
solver = pywraplp.Solver.CreateSolver("CBC")

inf = solver.infinity()

tIndex = input_data["simData"]["tIndex"] # number of timeslots
dt = input_data["simData"]["dt"] # time interval in hour

# Create datetime array
startTime = input_data["simData"]["startTime"].strftime("%Y-%m-%d %H:%M:%S")
tIndex = input_data["simData"]["tIndex"]
timestamp = pd.date_range(startTime, periods=tIndex, freq=str(dt * 60) + "min")
time = [timestamp[i].strftime("%Y-%m-%d %H:%M:%S") for i in range(len(timestamp))]

time_s = timeit.default_timer()

In [27]:
timeInterval

Timedelta('0 days 01:00:00')

In [98]:

# Add timeseries variables
vGrid = [solver.NumVar(lb=-inf, ub=inf, name=f"vGrid_{i}") for i in range(tIndex)]
vBattPower = [solver.NumVar(lb=-inf, ub=inf, name=f"vBattPower_{i}") for i in range(tIndex)]
vCharge = [solver.NumVar(lb=-inf, ub=0, name=f"vCharge_{i}") for i in range(tIndex)]
vDischarge = [solver.NumVar(lb=0, ub=inf, name=f"vDischarge_{i}") for i in range(tIndex)]
vChargeStatus = [solver.BoolVar(name=f"vChargeStatus_{i}") for i in range(tIndex)]
vSOC = [solver.NumVar(lb=0, ub=1, name=f"vSOC_{i}") for i in range(tIndex)]

# Add constraints
for i in range(tIndex):
    t = time[i]

    # Grid constraints
    solver.Add(vGrid[i] == input_data["market"]["alberta_internal_load"].get(t, 0) - input_data["market"]["solar_generation"].get(t, 0) -
               input_data["market"]["wind_generation"].get(t, 0) - vBattPower[i])  # Eqn. 1
    solver.Add(vGrid[i] <= input_data["grid"]["max_buy_power"])  # Eqn. 2
    solver.Add(vGrid[i] >= -input_data["grid"]["max_sell_power"])  # Eqn. 2
    solver.Add(input_data["market"]["alberta_internal_load"].get(t, 0) - input_data["market"]["solar_generation"].get(t, 0) -
               input_data["market"]["wind_generation"].get(t, 0) - (vDischarge[i] + vCharge[i]) <=
               input_data["grid"]["max_import_power"])  # Eqn. 3
    solver.Add(input_data["market"]["alberta_internal_load"].get(t, 0) - input_data["market"]["solar_generation"].get(t, 0) -
               input_data["market"]["wind_generation"].get(t, 0) - (vDischarge[i] + vCharge[i]) >=
               -input_data["grid"]["max_export_power"])  # Eqn. 3
    # Battery constraints
    solver.Add(vBattPower[i] == vCharge[i] + vDischarge[i])  # Eqn. 4
    solver.Add(vCharge[i] >= -input_data["batt"]["max_charge_rate"] * vChargeStatus[i])  # Eqn. 5(a)
    solver.Add(vDischarge[i] <= input_data["batt"]["max_discharge_rate"] * (1 - vChargeStatus[i]))  # Eqn. 5(b)
    if i == 0:
        solver.Add(vSOC[i] == input_data["batt"]["initial_soc"] - dt / input_data["batt"]["capacity"] *
                   (vCharge[i] * (input_data["batt"]["charge_eff"]) +
                    vDischarge[i] / (input_data["batt"]["discharge_eff"])))  # Eqn. 6
    else:
        solver.Add(vSOC[i] == vSOC[i-1] - dt / input_data["batt"]["capacity"] *
                   (vCharge[i] * (input_data["batt"]["charge_eff"]) +
                    vDischarge[i] / (input_data["batt"]["discharge_eff"])))  # Eqn. 6
    solver.Add(vSOC[i] >= input_data["batt"]["min_soc"])  # Eqn. 7
    solver.Add(vSOC[i] <= input_data["batt"]["max_soc"])  # Eqn. 7



In [99]:
# Add objective
obj = 0
#obj += sum(-[vBattPower[i] * input_data["market"]["pool_price"][time[i]] * dt for i in range(tIndex)])
for i in range(tIndex):
    t = time[i]
    pool_price = input_data["market"]["pool_price"].get(t, 0)
    #pool_price = input_data["market"]["pool_price"].get(t, 0)  # Use .get() to handle missing keys
    obj += -vBattPower[i] * pool_price * dt  # Accumulate the objective function
solver.Minimize(obj)

solver.Minimize(obj)

status = solver.Solve()

time_e = timeit.default_timer()
runTime = round(time_e - time_s, 4)

if status == solver.OPTIMAL or status == solver.FEASIBLE:
    print("Solution is found.")
    print("Number of variables =", solver.NumVariables())
    print("Number of constraints =", solver.NumConstraints())
    print("Computation time = ", runTime)
    
    # Extract solution values
    excelWriter = pd.ExcelWriter('D:/Python Projects/git_projects/BESS/Battery Optimization/results/Result.xlsx', engine='xlsxwriter')
    
    
    objValue = round(solver.Objective().Value() / 100, 2)
    
    objValueDF = pd.DataFrame.from_dict({"obj_value": objValue}, orient="index", columns=["Total Cost of Importing Power ($)"])
    
    result = list(zip([round(vGrid[i].solution_value(), 4) for i in range(tIndex)], 
                      [round(vBattPower[i].solution_value(), 4) for i in range(tIndex)],
                      [round(vCharge[i].solution_value(), 4) for i in range(tIndex)],
                      [round(vDischarge[i].solution_value(), 4) for i in range(tIndex)],
                      [round(vSOC[i].solution_value(), 4) for i in range(tIndex)],
                      [int(vChargeStatus[i].solution_value()) for i in range(tIndex)]
                      ))
    resultDF = pd.DataFrame(result, index=timestamp, columns=["Grid Power Flow (MW)", "Battery Output (MW)", "Charging Power (MW)", "Discharging Power (MW)", "State-of-charge (SOC)", "Charge Status"])
    
    objValueDF.to_excel(excelWriter, sheet_name='Cost')
    resultDF.to_excel(excelWriter, sheet_name='Operation')
    excelWriter.close()
else:
    print("Solution cannot be found.")

Solution is found.
Number of variables = 288
Number of constraints = 528
Computation time =  14.7294


In [100]:
resultDF

,Grid Power Flow (MW),Battery Output (MW),Charging Power (MW),Discharging Power (MW),State-of-charge (SOC),Charge Status
2024-12-30 00:00:00,9571.9966,0.0,0.0,0.0,0.5,0
2024-12-30 01:00:00,9428.0479,0.0,0.0,0.0,0.5,0
2024-12-30 02:00:00,9292.4505,0.0,0.0,0.0,0.5,0
2024-12-30 03:00:00,9231.0727,0.0,0.0,0.0,0.5,0
2024-12-30 04:00:00,9244.5748,0.0,0.0,0.0,0.5,0
2024-12-30 05:00:00,9500.2355,0.0,0.0,0.0,0.5,0
2024-12-30 06:00:00,9740.9515,0.0,0.0,0.0,0.5,0
2024-12-30 07:00:00,10095.5059,0.0,0.0,0.0,0.5,0
2024-12-30 08:00:00,10380.2987,0.0,0.0,0.0,0.5,0
2024-12-30 09:00:00,10383.9907,0.0,0.0,0.0,0.5,0
